In [ ]:
import requests
from requests import Response
import time

from pydantic import RootModel

from io import StringIO
from csv import DictReader
from pprint import pprint

class bikeOfSite:
    pass

class bikeOfSites(RootModel):
    root:list[dict]

    def __iter__(self):
        return iter(self.root)
    
    def __getitem__(self, idx):
        return self.root[idx]

def get_response() -> Response | None:
    url_path:str = 'https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json'

    count:int = 0
    limit:int = 3
    delay:int = 5

    while True and count < limit:
        try:
            print(f'執行第 {count + 1} 次下載:')
            response:Response = requests.get(url_path, timeout=15)
            response.raise_for_status()
        except Exception as e:
            count += 1
            print('連線失敗。')
            print(e)

            if count < limit:
                print(f'等待 {delay} 秒後再次嘗試。')
                time.sleep(delay)
        else:
            if response.status_code != 200:
                count += 1
                print('下載失敗。')
                print(response.reason)
                
                if count < limit:
                    print(f'等待 {delay} 秒後再次嘗試。')
                    time.sleep(delay)
            else:
                break
    
    if count >= limit:
        print(f'失敗已達 {limit} 次，不再執行下載動作。')
        return None
    else:
        print('下載成功')
        return response
    
############################

def csvToList(raw:str) -> list[dict] | None:
    file:StringIO = StringIO(raw)

    try:
        reader:DictReader = DictReader(file)
        data:list[dict] = list(reader)
        return data
    except Exception as e:
        print('預想外的格式。')
        return None
    finally:
        file.close()

############################

def jsonToList(raw:str) -> list[dict] | None:
    pass

def main():
    response:Response | None = get_response()

    if not response:
        return
    
    rawString:str = response.text
    data:list[dict] | None = jsonToList(rawString)

    if not data:
        return
    
    pprint(data)

main()